In [1]:
# importing the neccessary libraries for this work
import json
import random

In [17]:
# opening the files we'll be putting the data into
min_act = open("min_act.ndjson","a", encoding='utf-8')
mod_act = open("mod_act.ndjson","a", encoding='utf-8')
ver_act = open("ver_act.ndjson","a", encoding='utf-8')
ext_act = open("ext_act.ndjson","a", encoding='utf-8')

In [2]:
%%time
# opening the ndjson file to make a dictionary of username classes
uname_dict = {}
count_urls = 0

# getting username activities
with open("post.ndjson", encoding="utf-8") as f_in:
    for line in f_in:
        line_json = json.loads(line)
        
        # checking the username is in the json; if not it's not useful for us
        if "un" in line_json:
            # checking if the line has a body OR a url
            # this matters b/c of the way the data is structured
            # sometimes we don't have any body because the body consisted solely of a URL which was removed as part of pre-proc
            if "bo" or "url" in line_json:
                uname = line_json["un"]
                
                if uname not in uname_dict:
                    uname_dict.setdefault(uname, 1)

                else:
                    uname_dict[uname] += 1


CPU times: total: 2min 39s
Wall time: 2min 40s


In [5]:
%%time
bucket_count = {"min_act":0, "mod_act":0, "ver_act":0, "ext_act":0}

for uname in uname_dict:
    # getting the count for the numbers of users in each bucket
            if uname_dict[uname] < 100:
                bucket_count["min_act"] += 1
                
            elif uname_dict[uname] >= 100 and uname_dict[uname] < 1000:
                bucket_count["mod_act"] += 1
                                     
            elif uname_dict[uname] >= 1000 and uname_dict[uname] < 10000:
                bucket_count["ver_act"] += 1
                
            elif uname_dict[uname] > 10000:
                bucket_count["ext_act"] += 1
                

CPU times: total: 812 ms
Wall time: 819 ms


In [6]:
bucket_count

{'min_act': 2346024, 'mod_act': 21247, 'ver_act': 2325, 'ext_act': 97}

In [19]:
# defining function to choose the things we care about
def line_clean(line_json):
    
    line_data = {}

    if "t" in line_json: # t = type of item
        line_data.setdefault("t",line_json["t"])
    
    if "id" in line_json: # id = uid for item
        line_data.setdefault("id",line_json["id"])

    if "cd" in line_json: # cd = creation date -> yyyymmdd
        line_data.setdefault("cd",line_json["cd"])

    if "c" in line_json: # c = creation date -> unix epoch
        line_data.setdefault("c",line_json["c"])

    if "u" in line_json: # u = uid for user
        line_data.setdefault("u",line_json["u"])

    if "un" in line_json: # un = username for user
        line_data.setdefault("un",line_json["un"])
    
    if "dmn" in line_json: # dmn = list of domains
        line_data.setdefault("dmn",line_json["dmn"])
    
    if "bo" in line_json: # bo = body of text
        line_data.setdefault("bo",line_json["bo"])
    
    return line_data

In [20]:
%%time
# splitting the posts up
posts_written = {"min_act":0, "mod_act":0, "ver_act":0, "ext_act":0}

# opening the ndjson file
with open("post.ndjson", encoding="utf-8") as f_in:
    for line in f_in:
        line_json = json.loads(line)
        
        
        # filtering out lines lacking a uname; if there's no uname we can't really work with it
        if "un" in line_json:
            uname = line_json["un"]          
            
            # pulling the elements we care about from the line_json
            line_json = line_clean(line_json)
                        
            # filtering lines into appropriate files
            if uname_dict[uname] < 100:
                
                
                        posts_written["min_act"] += 1
                        
                        # using json.dumps to create json object & adding a newline to make ndjson
                        min_act.write(json.dumps(line_json)+"\n")
                
            elif uname_dict[uname] >= 100 and uname_dict[uname] < 1000:
                
                        mod_act.write(json.dumps(line_json)+"\n")
                        posts_written["mod_act"] += 1

            
            elif uname_dict[uname] >= 1000 and uname_dict[uname] < 10000:
                
                        ver_act.write(json.dumps(line_json)+"\n")
                        posts_written["ver_act"] += 1

                
            elif uname_dict[uname] > 10000:
                    
                        ext_act.write(json.dumps(line_json)+"\n")
                        posts_written["ext_act"] += 1

CPU times: total: 5min 25s
Wall time: 5min 26s


In [21]:
print(posts_written)

{'min_act': 7272338, 'mod_act': 5647816, 'ver_act': 5689503, 'ext_act': 1984388}


In [22]:
%%time
# checking the comments

comments_written = {"min_act":0, "mod_act":0, "ver_act":0, "ext_act":0}

no_post_count = 0

with open("comment.ndjson", encoding="utf-8") as f_in:
    for line in f_in:
        line_json = json.loads(line)
        
        # pulling the elements we care about from the line_json
        line_json = line_clean(line_json)
            
        # filtering out lines lacking a uname; if there's no uname we can't really work with it
        if "un" in line_json:
            uname = line_json["un"]          
            
            # ensuring that the uname is actually tracked by us (meaning they posted either text or a url)
            if uname in uname_dict:
                
                # filtering lines into appropriate files
                if uname_dict[uname] < 100:

                            comments_written["min_act"] += 1
                            
                            # using json.dumps to create json object & adding a newline to make ndjson
                            min_act.write(json.dumps(line_json)+"\n")

                elif uname_dict[uname] >= 100 and uname_dict[uname] < 1000:

                            mod_act.write(json.dumps(line_json)+"\n")
                            comments_written["mod_act"] += 1


                elif uname_dict[uname] >= 1000 and uname_dict[uname] < 10000:

                            ver_act.write(json.dumps(line_json)+"\n")
                            comments_written["ver_act"] += 1


                elif uname_dict[uname] > 10000:

                            ext_act.write(json.dumps(line_json)+"\n")
                            comments_written["ext_act"] += 1
                        
            # else they didn't have any posts so we can't really work with their comment under our system
            else:
                no_post_count += 1

CPU times: total: 16min 45s
Wall time: 16min 46s


In [23]:
print(comments_written)

{'min_act': 18033602, 'mod_act': 12062604, 'ver_act': 5516705, 'ext_act': 270038}


In [24]:
min_act.close()
mod_act.close()
ver_act.close()
ext_act.close()

From the start, we can see that the levels of activity using posts as a metric for engagment don't bear out when applied to comments. Instead, what we see is that the minimally active users have the most comments out of any group. 